### Загружу библиотеки

In [ ]:
import pymorphy2
morph=pymorphy2.MorphAnalyzer()
import numpy as np
import datetime
random_state = 17
import matplotlib.pyplot as plt
import itertools
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import ExtraTreeClassifier
import pickle as pickle
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import sklearn.metrics
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')
# будем отображать графики прямо в jupyter'e
%config InlineBackend.figure_format = 'svg' 

import re
import pandas as pd
from collections import Counter
import xmltodict
from tqdm import tqdm
import pickle as pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer



### Предобработка текста и модель

In [ ]:
class myPreprocessor(object):
    
    global _names,_surnames, _secondnames 
    def __init__(self, _names):

        self._names = _names[0]
        self._surnames = _names[1]
        self._secondnames = _names[2]           

    def preprocess1(self, colname, text):
        if colname not in self._preprocess:
            return text
        func = self._preprocess[colname]
        if isinstance(text, pd.Series):
            return text.apply(func)
        return func(text)

    def preprocess(self, colname, text):
        
        global change_level_5
        def change_level_5(text, level, template):
            if text in level: return template
            return text_proc_3(text)    
    
    
        global replace_template
        def replace_template(row, target, template):
            row = row.split()
            for i in range(len(row)):
                if row[i].count(target) > 0:
                    row[i] = template
            row = text_proc_3(concat(row))
            return row
        
        global text_proc_3
        def text_proc_3(text):
            if text != '':
                text = re.sub(r"\s+", " ", text) # убрал лишние пробелы (\s+ Любой пробельный символ 1 и более вхождений шаблона)
                text = re.sub(r"^\s+", "", text) #  уберу первые пробелы в начале строки
                text = text.rstrip() #  уберу последний пробел в начале строки
            return text
        
        global cat_from_template2
        def cat_from_template2(text, level, template):
            row = text.split(' ')
            row = [change_level_5(x, level, template) for x in row]
            row = text_proc_3(concat(row))
            return row
        
        global cat_phrase
        def cat_phrase(phrase, text):
            if phrase in text:
                nomer_phrase = text.find(phrase)
                text = text[0:nomer_phrase:]
            return text
        
        global collapse_ne_2
        def collapse_ne_2(string):
            if string != '':
                string = string.split(' ')
                for i in range(len(string)-1):
                    if string[i] == 'не':
                        string[i] = string[i]+'_'+string[i+1]
                        string[i+1] = ''
                string0 = string[0]
                for i in range(1, len(string)):
                    string0 = string0 + ' ' + string[i]
                string = re.sub(r'\s+', ' ', string0)
            return string
    
        global compare   
        def compare(a, b):
            if a == b: return 1
            if a != b: return 0
        
        global got_counter
        def got_counter(per):
            counter = Counter()
            for i in range(len(per)):
                word = per[i]
                counter[word] += 1
            return counter

        global lemma     
        def lemma(txt):
            return morph.parse(txt)[0].normal_form
        
        global concat
        def concat(vec):
            return ' '.join(vec)
    
        global cat_from_template
        def cat_from_template(template, text_string):
            row = text_string.split()
            row = [x for x in row if x not in template]
            row = concat(row)
            return row
        
        global mynewpreprocessing4
        def mynewpreprocessing4(string, A = 3):
            if string != '':
                lev = ['\n', '\r', 'нестандартный','зно','подробное','описание','null','вопрос/проблема','информацию по',
                       'инциденту','вы можете увидеть в протоколе','добрый','день', 'коллеги', 'в работу поступила заявка',
                       'просим','помочь','спасибо', 'просьба','решении','ошибке','оказать','содействие','проблемы','по заявке',
                       'имеется','следующая','проблема','помогите','решить проблему','разбудить','прошу решить вопрос',
                       'как можно','очень','срочно','быстрее','табельный номер','здравствуйте',
                       ';', '=', '–', '*', '%', '<', '>', '"', '-', '«', '»', '(', ')']        
                string = replace_template(string, 'Cброс', 'Сброс')
                string = string.lower()
                string = string.replace('ё', 'е')
                string = replace_template(string, '@', 'почтовый_адрес')
                string = string.replace(':', '.')
                string = string.replace('!', '.')
                string = string.replace(',', '.')
                string = string.replace('.', ' . ')
                string = cat_from_template2(string, lev, ' ')
                string = cat_phrase('с уважением', string)
                string = string.replace(' . ', ' ')
                string = string.replace('№', '')
                string = string.replace('sd', '')
                string = string.replace('im', '')
                string = string.replace('\\', ' ')
                string = string.replace(' .', '. ')
                string = string.replace('•', '.')
                string = string.replace('.', ' ')
                string = re.sub('[^а-яА-Яa-zA-Z]', ' ', string) # заменил на пробел не буквы
                string = text_proc_3(string) # убрал лишние пробелы    
                string = text_proc_3(concat(list(  [lemma(j) for j in string.split() ] ) ))
                string = collapse_ne_2(string)
            
                # вытащим все англ слова и удалим лишние
                all_uniq = []
                row = [text_proc_3(re.sub('[^a-zA-Z]', ' ', string))]
                for j in row:
                    if j != '':
                        all_uniq.append(j)
            
                counter = got_counter(all_uniq)
                if counter != Counter():
                    res_df = pd.DataFrame.from_dict(counter, orient='index').reset_index().rename(columns={'index':'event', 0:'count'})
                    res_df = res_df.sort_values(by=['count'], ascending=False).reset_index(drop=True)       
                    levelB = [x for x in res_df[res_df['count'] < A]['event']]
                    levelB.sort()
                    string = text_proc_3(cat_from_template(levelB, string))
                    
                # вытащим все RUS слова и удалим лишние
                all_uniq = []
                row = [text_proc_3(re.sub('[^а-яА-Я_]', ' ', string))]
                for j in row:
                    if j != '':
                        all_uniq.append(j)
                
                counter = got_counter(all_uniq)
                if counter != Counter():
                    res_df = pd.DataFrame.from_dict(counter, orient='index').reset_index().rename(columns={'index':'event', 0:'count'})
                    res_df = res_df.sort_values(by=['count'], ascending=False).reset_index(drop=True) 
                    levelB = [x for x in res_df[res_df['count'] < A]['event']]
                    levelB.sort()
                    string = text_proc_3(cat_from_template(levelB, string))
                return string
            return string
    
        global devide_keys_values_3
        def devide_keys_values_3(wk, diction):
            global df
            if diction is not None:
                if type(diction) == str: print ('str', diction);
                if type(diction) != list and type(diction) != str:
                    for key, value in diction.items():
                        if type(value) == str: 
                            if len(wk) != 0:  w = wk + '-' + str(key)
                            if len(wk) == 0:  w = str(key) 
                            df[w] = str(value)
                        if type(value) == list:
                            for i in range(len(value)):            
                                if len(wk) != 0:  w = str(i) + wk + '-' + str(key)  #str(diction.keys()[0])
                                if len(wk) == 0:  w = str(i) + str(key)  #str(diction.keys()[0]) 
                                devide_keys_values_3( w, value[i])
                        if type(value) != str and type(value) != list: 
                            if len(wk) != 0:  w = wk + '-' + str(key)
                            if len(wk) == 0:  w = str(key) 
                            devide_keys_values_3( w, value)
                if type(diction) == list:
                    if len(wk) != 0:  w = wk + '-' + str(diction[0])
                    if len(wk) == 0:  w = str(diction[0]) 
                    for i in diction:
                        devide_keys_values_3( w, i)    
    
        global find_name_surname
        def find_name_surname(row):
            global _names, _surnames, _secondnames
            rez = -1
            st = ''
            txt = ''
            if len(row) == 3:
                sec = row[2][-3:]
                if sec == 'вна' or sec == 'вич' or sec == 'чна' or sec == 'ьич':
                    if row[1] in self._names:
                        if row[0] in self._surnames:
                            txt = row[0] + ' ' + row[1] + ' ' + row[2]
                            rez = 0; st = 'ОК'
                            self._secondnames.append(row[2])
                        else:
                            txt = row[0] + ' ' + row[1] + ' ' + row[2]
                            rez = 1; st = '+Ф'
                            self._surnames.append(row[0])
                            self._secondnames.append(row[2])
                    if row[1] not in self._names:
                        if row[0] in self._surnames: 
                            self._names.append(row[1])
                            self._secondnames.append(row[2])
                            rez = 2; st = '+И'
                        else:
                            if row[2][-4:] == 'овна':
                                if row[2][:-4] in self._names:
                                    self._names.append(row[1])
                                    self._surnames.append(row[0])
                                    self._secondnames.append(row[2])
                                    rez = 3; st = '+ИФ'
                                else:
                                    self._names.append(row[1])
                                    self._surnames.append(row[0])
                                    self._secondnames.append(row[2])
                                    rez = 4; st = '+ИФ-'
                if sec != 'вна' and sec != 'вич' and sec != 'чна' and sec != 'ьич':
                    if row[1][-3:] == 'вна' or row[1][-3:] == 'вич' or row[1][-3:] == 'чна':
                        self._secondnames.append(row[1])
                        if row[0] in self._names:
                            txt = row[2] + ' ' + row[0] + ' ' + row[1]
                            rez = 5; st = 'ИОФ-ФИО'
                            self._surnames.append(row[2])                        
            return txt, rez, st
        
        global preptext_en
        def preptext_en(text):
            text = text.replace('\n', ' ').replace('\r', ' ').replace(',', ' ').replace('>',' ' )
            text = text.replace('ё', 'е')
            text = re.sub('[^а-яА-Яa-zA-Z]', ' ', text) # заменил на пробел не буквы
            text = text.lower()
            
            lev = ['нестандартный','зно','подробное','описание','null','вопрос/проблема','информацию по',
               'инциденту','вы можете увидеть в протоколе','добрый','день', 'коллеги', 'в работу поступила заявка',
               'просим','помочь','спасибо', 'просьба','решении','ошибке','оказать','содействие','проблемы','по заявке',
               'имеется','следующая','проблема','помогите','решить проблему','разбудить','прошу решить вопрос',
               'как можно','очень','срочно','быстрее','табельный номер','здравствуйте',
               ';', '=', '–', '*', '%', '<', '>', '"', '-', '«', '»', '(', ')'] 
            
            text = cat_from_template2(text, lev, ' ')
            
            text = re.sub(r"\s+", " ", text) # убрал лишние пробелы (\s+ Любой пробельный символ 1 и более вхождений шаблона)
            text = re.sub(' \w ', ' ', text) # Любая цифра или буква
            text = re.sub(r"^\s+", "", text) #  уберу первые пробелы в начале строки
            text = text.rstrip() #  уберу последний пробел в начале строки
            return text
               
        global mynewpreprocessing3
        def mynewpreprocessing3(db, A = 3):
            lev = ['\n', '\r', 'нестандартный','зно','подробное','описание','null','вопрос/проблема','информацию по',
                   'инциденту','вы можете увидеть в протоколе','добрый','день', 'коллеги', 'в работу поступила заявка',
                   'просим','помочь','спасибо', 'просьба','решении','ошибке','оказать','содействие','проблемы','по заявке',
                   'имеется','следующая','проблема','помогите','решить проблему','разбудить','прошу решить вопрос',
                   'как можно','очень','срочно','быстрее','табельный номер','здравствуйте',
                   ';', '=', '–', '*', '%', '<', '>', '"', '-', '«', '»', '(', ')']
            db = [replace_template(x, 'Cброс', 'Сброс') for x in db]
            db = [x.lower() for x in db]
            db = [x.replace('ё', 'е') for x in db]
            db = [replace_template(x, '@', 'почтовый_адрес') for x in db] # ЗАМЕНИЛ почтовый адрес на темплейт
            db = [x.replace(':', '.') for x in db]
            db = [x.replace('!', '.') for x in db]
            db = [x.replace(',', '.') for x in db]
            db = [x.replace('.', ' . ') for x in db]
            db = [cat_from_template2(x, lev, ' ') for x in db]
            db = [cat_phrase('с уважением', x) for x in db]
            db = [x.replace(' . ', ' ') for x in db]
            db = [x.replace('№', '') for x in db]
            db = [x.replace('sd', '') for x in db]
            db = [x.replace('im', '') for x in db]
            db = [x.replace('\\', ' ') for x in db] 
            db = [x.replace(' .', '. ') for x in db]
            db = [x.replace('•', '.') for x in db]
            db = [x.replace('.', ' ') for x in db]
            db = [re.sub('[^а-яА-Яa-zA-Z]', ' ', x)  for x in db] # заменил на пробел не буквы
            db = [text_proc_3(x) for x in db] # убрал лишние пробелы    
            db = [text_proc_3(concat(list(  [lemma(j) for j in x.split() ] ) )) for x in db]
            db = [collapse_ne_2(x) for x in db]
           
            # вытащим все англ слова и удалим лишние
            all_uniq = []
            for i in (range(len(db))):
                row = [text_proc_3(re.sub('[^a-zA-Z]', ' ', x)) for x in db[i].split(' ') if text_proc_3(x) != '']
                for j in row:
                    if j != '':
                        all_uniq.append(j)
            
            counter = got_counter(all_uniq)
            if counter != Counter():
                res_df = pd.DataFrame.from_dict(counter, orient='index').reset_index().rename(columns={'index':'event', 0:'count'})
                res_df = res_df.sort_values(by=['count'], ascending=False).reset_index(drop=True)       
                levelB = [x for x in res_df[res_df['count'] < A]['event']]
                levelB.sort()
                db = [text_proc_3(cat_from_template(levelB, x)) for x in db]
                
            # вытащим все RUS слова и удалим лишние
            all_uniq = []
            for i in (range(len(db))):
                row = [text_proc_3(re.sub('[^а-яА-Я_]', ' ', x)) for x in db[i].split(' ') if text_proc_3(x) != '']
                for j in row:
                    if j != '':
                        all_uniq.append(j)
            
            counter = got_counter(all_uniq)
            if counter != Counter():
                res_df = pd.DataFrame.from_dict(counter, orient='index').reset_index().rename(columns={'index':'event', 0:'count'})
                res_df = res_df.sort_values(by=['count'], ascending=False).reset_index(drop=True) 
                levelB = [x for x in res_df[res_df['count'] < A]['event']]
                levelB.sort()
                db = [text_proc_3(cat_from_template(levelB, x)) for x in db]
                #db = [text_proc_3(cat_from_template(trash, x)) for x in tqdm(db)]
            return db
        
        global change_nan
        def change_nan(x):
            if str(x) == 'nan':
                return '0'
            else:
                return str(x)
            
        global drop2
        def drop2(text, tr):
            if text in tr: return ''
            return text

        
        global mynewpreprocessing_opt
        def mynewpreprocessing_opt(string_opt):
            global _names, _surnames, _secondnames, df
            
            if string_opt != '':
                if string_opt.count('<') == 0: 
                    return string_opt
                if string_opt.count('<') != 0:
#                    print (string_opt)
                    doc = xmltodict.parse(string_opt)
                    wk = []
                    df = pd.DataFrame({'num': ['0']})
                    devide_keys_values_3(wk, doc)        
                    mtr = df.copy()        
                    mtr.fillna('',inplace=True)        
                    del mtr['num']  
                    mtr = mtr.reset_index(drop=True)        
                    nmsval = []
                    for i in mtr.columns:
                        if len(i) > 10:
                            if i[-10:] == 'text-#text' or i[-25:] == 'form-select-option-@label' or i[-24:] == 'form-checkbox-@sbcommand' or i[-18:] == 'form-select-@label':
                                nmsval.append(i)        
                    mtrval = mtr[nmsval].copy()
                    mtrval = [x for x in mtrval.iloc[0,:]]
                    # почистил матрицу с текстовыми колонками от лишних символов
                    delete = ['div', 'style', 'color', 'red', 'strong']
                    for j in range(len(mtrval)):
                        mtrval[j] = preptext_en(change_nan(mtrval[j]))
                        for i in range(len(delete)):
                            if str(mtrval[j]).count(delete[i]) > 0:
                                mtrval[j] = ''
                    mtrval = [x for x in mtrval if x != '']
#                    for j in tqdm(mtrval.columns):
#                        u = [x for x in mtrval[j].unique()]
#                        c = [len(mtrval.loc[mtrval[j] == x, j]) for x in u]
#                        r = [u[x] for x in range(len(c)) if c[x] > 2]
#                        for i in range(len(mtrval[j])):
#                            for x in range(len(r)):
#                                if mtrval.loc[i, j] == r[x]:
#                                    mtrval.loc[i, j] = ''
                    vec = []
                    for j in range(len(mtrval)):
                        vector = [find_name_surname(mtrval[j].split())]
                        d0 = [ vector[x] for x in range(len(vector)) if vector[x][1] != -1]
                        vec = vec + d0
                        
                    self._names = list(set(self._names)); self._names.sort()
                    self._surnames = list(set(self._surnames)); self._surnames.sort()
                    self._secondnames = list(set(self._secondnames)); self._secondnames.sort()
                    _trash = self._surnames + self._names + self._secondnames
                    
                    # удалю все фамилии и треш из полей OPTIONS
                    for k in range(len(mtrval)):
                        row = mtrval[k].split()
                        vec = [drop2(j, _trash) for j in row ]
                        vec = text_proc_3(concat(vec))
                        mtrval[k] = collapse_ne_2(vec)       
                    rez = ''
                    for k in range(len(mtrval)):
                        rez = rez + ' ' + mtrval[k]  
                    rez = text_proc_3(rez)
                    return rez    
            return string_opt

        global mynewpreprocessing_opt_mega
        def mynewpreprocessing_opt_mega(string_opt):
            global _names, _surnames, _secondnames, df
            
            if string_opt != '':
                if string_opt.count('<') == 0: 
                    return string_opt
                if string_opt.count('<') != 0:
#                    print (string_opt)
                    doc = xmltodict.parse(string_opt)
                    wk = []
                    df = pd.DataFrame({'num': ['0']})
                    devide_keys_values_3(wk, doc)        
                    mtr = df.copy()        
                    mtr.fillna('',inplace=True)        
                    del mtr['num']  
                    mtr = mtr.reset_index(drop=True)        
                    nmsval = []
                    for i in mtr.columns:
                        if len(i) > 10:
                            if i[-10:] == 'text-#text' or i[-25:] == 'form-select-option-@label' or i[-24:] == 'form-checkbox-@sbcommand' or i[-18:] == 'form-select-@label':
                                nmsval.append(i)        
                    mtrval = mtr[nmsval].copy()
                    mtrval = [x for x in mtrval.iloc[0,:]]
                    # почистил матрицу с текстовыми колонками от лишних символов
                    delete = ['div', 'style', 'color', 'red', 'strong']
                    for j in range(len(mtrval)):
                        mtrval[j] = preptext_en(change_nan(mtrval[j]))
                        for i in range(len(delete)):
                            if str(mtrval[j]).count(delete[i]) > 0:
                                mtrval[j] = ''
                    mtrval = [x for x in mtrval if x != '']
                    for j in tqdm(mtrval.columns):
                        u = [x for x in mtrval[j].unique()]
                        c = [len(mtrval.loc[mtrval[j] == x, j]) for x in u]
                        r = [u[x] for x in range(len(c)) if c[x] > 2]
                        for i in range(len(mtrval[j])):
                            for x in range(len(r)):
                                if mtrval.loc[i, j] == r[x]:
                                    mtrval.loc[i, j] = ''
                    vec = []
                    for j in range(len(mtrval)):
                        vector = [find_name_surname(mtrval[j].split())]
                        d0 = [ vector[x] for x in range(len(vector)) if vector[x][1] != -1]
                        vec = vec + d0
                        
                    self._names = list(set(self._names)); self._names.sort()
                    self._surnames = list(set(self._surnames)); self._surnames.sort()
                    self._secondnames = list(set(self._secondnames)); self._secondnames.sort()
                    _trash = self._surnames + self._names + self._secondnames
                    
                    # удалю все фамилии и треш из полей OPTIONS
                    for k in range(len(mtrval)):
                        row = mtrval[k].split()
                        vec = [drop2(j, _trash) for j in row ]
                        vec = text_proc_3(concat(vec))
                        mtrval[k] = collapse_ne_2(vec)       
                    rez = ''
                    for k in range(len(mtrval)):
                        rez = rez + ' ' + mtrval[k]  
                    rez = text_proc_3(rez)
                    return rez    
            return string_opt



        
        global string_replace_xml
        def string_replace_xml(string):    
            s1 = string.replace('&lt___div', ' ')
            s1 = s1.replace('&#xD___', ' ')
            s1 = s1.replace('&quot___', ' ')
            s1 = s1.replace('&#**___', ' ')
            s1 = s1.replace('&lt___', ' ')
            s1 = s1.replace('&gt___', ' ')
            s1 = s1.replace('&#', ' ')
            s1 = s1.replace('*', ' ')
            s1 = s1.replace('#', ' ')
            s1 = s1.replace('.', ' ')
            s1 = s1.replace(',', ' ')
            s1 = s1.replace('!', ' ')
            s1 = s1.replace('$', ' ')
            s1 = s1.replace('%', ' ')
            return s1
                
        
        global mynewpreprocessing_opt_series
        def mynewpreprocessing_opt_series(list_opt):
            global _names, _surnames, _secondnames, df
            doc = xmltodict.parse(list_opt[0])
            wk = []
            df = pd.DataFrame({'num': ['0']})
            devide_keys_values_3(wk, doc)
            mtr = df.copy()
            for j in tqdm(range(1, len(list_opt))):
                p = list_opt[j]
                if str(p) != 'nan' and str(p) != '':
                    s1 = string_replace_xml(p)
                    doc = xmltodict.parse(s1)
                    wk = []
                    df = pd.DataFrame({'num': ['0']})
                    devide_keys_values_3(wk, doc)        
                    for i in range(df.shape[1]): mtr.loc[j,df.columns[i]] = df.iloc[0,i]
                else:
                    for i in range(df.shape[1]): mtr.loc[j,df.columns[i]] = ''
            mtr.fillna('',inplace=True)
            del mtr['num']  
            mtr = mtr.reset_index(drop=True)
            nmsval = []
            for i in mtr.columns:
                if len(i) > 10:
                    if i[-10:] == 'text-#text' or i[-25:] == 'form-select-option-@label' or i[-24:] == 'form-checkbox-@sbcommand' or i[-18:] == 'form-select-@label':
                        nmsval.append(i)
            mtrval = mtr[nmsval].copy()
            for j in (mtrval.columns):
                mtrval[j] = [preptext_en(change_nan(x)) for x in mtrval[j]] 
            
            for j in tqdm(mtrval.columns):
                u = [x for x in mtrval[j].unique()]
                c = [len(mtrval.loc[mtrval[j] == x, j]) for x in u]
                r = [u[x] for x in range(len(c)) if c[x] > 2]
                for i in range(len(mtrval[j])):
                    for x in range(len(r)):
                        if mtrval.loc[i, j] == r[x]:
                            mtrval.loc[i, j] = ''
            
            vec = []
            for k in tqdm(range(mtrval.shape[1])):
                vector = [find_name_surname(x.split()) for x in mtrval.iloc[:,k]]
                d0 = [ vector[x] for x in range(len(vector)) if vector[x][1] != -1]
                vec = vec + d0   
            self._names = list(set(self._names)); self._names.sort()
            self._surnames = list(set(self._surnames)); self._surnames.sort()
            self._secondnames = list(set(self._secondnames)); self._secondnames.sort()
            _trash = self._surnames + self._names + self._secondnames
            for k in tqdm(range(mtrval.shape[0])):
                for i in (range(mtrval.shape[1])):
                    row = mtrval.iloc[k,i].split()
                    vec = [drop2(j, _trash) for j in row ]
                    vec = text_proc_3(concat(vec))
                    mtrval.iloc[k,i] = collapse_ne_2(vec)
            mtrval['rez'] = mtrval.iloc[:,0]
            cols = [x for x in mtrval.columns[1:-1]]
            for nm in tqdm(cols):
                mtrval['rez'] = mtrval['rez'] + ' ' + mtrval[nm]
            list_opt = [text_proc_3(x) for x in mtrval['rez']]
            return list_opt                

# тут функционал )          
        if isinstance(text, pd.Series):
            if colname == 'options_mega': 
                if len([1 for x in text if x.count('<') != 0]) == 0: 
                    return text
                if len([1 for x in text if x.count('<') != 0]) == len(text): 
                    return mynewpreprocessing_opt_mega(text)
                if len([1 for x in text if x.count('<') != 0]) < len(text): 
                    ind = [x for x in range(len(text)) if text[x].count('<') != 0]
                    text_opt = [x for x in text if x.count('<') != 0]
                    text_opt = mynewpreprocessing_opt_mega(text_opt)
                    for i in range(len(ind)):
                        text[ind[i]] = text_opt[i]
                return text
            
            if colname == 'options': 
                if len([1 for x in text if x.count('<') != 0]) == 0: 
                    return text
                if len([1 for x in text if x.count('<') != 0]) == len(text): 
                    return mynewpreprocessing_opt_series(text)
                if len([1 for x in text if x.count('<') != 0]) < len(text): 
                    ind = [x for x in range(len(text)) if text[x].count('<') != 0]
                    text_opt = [x for x in text if x.count('<') != 0]
                    text_opt = mynewpreprocessing_opt_series(text_opt)
                    for i in range(len(ind)):
                        text[ind[i]] = text_opt[i]
                return text
            if colname != 'options':                     
                return mynewpreprocessing3(text, A = 3)
        if isinstance(text, str):        
            if colname != 'options':                     
                return mynewpreprocessing4(text, A = 3)
            if colname == 'options': 
                return mynewpreprocessing_opt(text)                
        return text

    def process(self, text):
        if isinstance(text, pd.Series):
            text = text.tolist()
        if isinstance(text, str):
            text = [text]
        if isinstance(text, list):
            string = [mynewpreprocessing4(x, A = 3) for x in text]
            return string
        return text


    def load(self):
        misprints_arr = []

    def dump(self):
        misprints_arr = []
        
        # self._preprocess = None
        # self._names = None
        # self._processing_dict = None

        # return self.dict_dump()

class One_vs_all():
    def __init__(self, train, y_train, regressor, vectorizer, preprocessor, random_state=42, params={}, num_boost_round=500):
        self.train = train
        self.y_train = y_train
        self.random_state = random_state
        self.params = params
        self.num_boost_round = num_boost_round
        self.y_dummied = ''
        self.one_vs_all_models = {}

        self.vectorizer = vectorizer
        self.regressor = regressor
        self.is_preprocessed = False
        self.cv_dict = {}

        self.preprocessor = preprocessor
        self.preprocessed_data = None
        self.processed_data = None

    def set_preprocessor(self, preprocessor):
        self.preprocessor = preprocessor

    def remove_zero_rows(self, matr):
        nonzero_row_indice, _ = matr.nonzero()
        unique_nonzero_indice = np.unique(nonzero_row_indice)
        return matr[unique_nonzero_indice], unique_nonzero_indice

    def preprocess_data(self, data):
        data = pd.DataFrame(data)
        if self.preprocessor is not None:
            for colname in data.columns:
                data[colname] = self.preprocessor.preprocess(colname, data[colname])
        return data.apply(lambda x: ' '.join(x.apply(str).tolist()), axis=1)

    def preprocess_fit(self):
        if self.is_preprocessed:
            return
        self.vectorizer = self.vectorizer.fit(self.train)
        train_tfidf = self.vectorizer.transform(self.train)

        self.train_tfidf, not_zero_rows = self.remove_zero_rows(train_tfidf)

        self.y_train = self.y_train.iloc[not_zero_rows]

        self.y_dummied = pd.get_dummies(self.y_train)
        for class_name in self.y_dummied.columns:
            if self.y_dummied[class_name].sum() < 10:
                self.y_dummied.drop(class_name, axis=1, inplace=True)

        self.is_preprocessed = True

    def model_fit(self):
        for class_name in self.y_dummied.columns:
            y_train = self.y_dummied[class_name]
            temp_params = self.params
            scale = len(y_train) / y_train.sum()
            if self.regressor == xgb:
                dtrain = xgb.DMatrix(self.train_tfidf, y_train)
                temp_params["scale_pos_weight"] = scale
                class_model = xgb.train(temp_params, dtrain, self.num_boost_round)
            else:
                dtrain = self.train_tfidf
                params = self.params
                params['random_state'] = self.random_state
                class_model = self.regressor(**params).fit(dtrain, y_train)

            self.one_vs_all_models[class_name] = class_model

    def cv_model(self):
        if self.regressor != xgb:
            print('cv_model supported only for xgboost')
            return None

        self.preprocess_fit()
        if len(self.cv_dict) > 0:
            return self.cv_dict

        cv_dict = {}
        for class_name in self.y_dummied.columns:
            print('cv for ', class_name)
            y_train = self.y_dummied[class_name]
            if self.regressor == xgb:
                dtrain = xgb.DMatrix(self.train_tfidf, y_train)
                cv_info = xgb.cv(self.params, dtrain, num_boost_round=self.num_boost_round, nfold=5, stratified=True, metrics='logloss', verbose_eval=100)
                '''else:
                params = self.params
                params['random_state'] = self.random_state
                scoresSSS = cross_val_score(self.regressor(**params), self.train_tfidf, y_train, cv=5, n_jobs=-1)
                cv_info = np.average(scoresSSS)
                #print(cv_info)'''
            cv_dict[class_name] = cv_info
        self.cv_dict = cv_dict
        return cv_dict

    def clear(self):
        del self.train_tfidf
        del self.y_dummied

    def mypredict_single(self, text):
        result = {}
        if isinstance(text, pd.Series):
            text = text.tolist()
        if isinstance(text, str):
            text = [text]
        if isinstance(text, list):
### 27_06_18 start
            text = self.preprocessor.preprocess('DESCRIPTION', text)
### 27_06_18 end
            test = self.vectorizer.transform(text)
#            y_pred = self.regressor.predict(test)
            res = self.regressor.predict_proba(test)
            res = pd.DataFrame(res)
            res.columns = self.regressor.classes_
            result['res_class'] = str(res.idxmax(axis=1).iloc[0])
            result['res_prob'] = res.max(axis=1).iloc[0]
            return result
        return {'res_class': '', 'res_prob': 0.0}


    def mypredict_double(self,text):
        result = {}
        desr = text[0]
        opt = text[1] 
        desr = self.preprocessor.preprocess('DESCRIPTION', desr)
        opt = self.preprocessor.preprocess('options', opt)
        text = desr + ' ' + opt
        if isinstance(text, pd.Series):
            text = text.tolist()
        if isinstance(text, str):
            text = [text]
        if isinstance(text, list):
            test = self.vectorizer.transform(text)
#            y_pred = self.regressor.predict(test)
            res = self.regressor.predict_proba(test)
            res = pd.DataFrame(res)
            res.columns = self.regressor.classes_
            result['res_class'] = str(res.idxmax(axis=1).iloc[0])
            result['res_prob'] = res.max(axis=1).iloc[0]
            return result
        return {'res_class': '', 'res_prob': 0.0}

 

    def predict_single(self, text):
        if self.preprocessor is not None:
            text = self.preprocessor.process(text)
        else:
            text = [text]
        print(text)
        test = text
        test_tfidf = self.vectorizer.transform(test)
        if self.regressor == xgb:
            dtest = xgb.DMatrix(test_tfidf)
        else:
            dtest = test_tfidf
        result = {}
        if type(self.regressor).__name__ in ['type', 'module']:
            for class_name in self.one_vs_all_models:
                model = self.one_vs_all_models[class_name]
                proba = model.predict(dtest)[0].apply(str)
                if proba > 0.5:
                    result[class_name] = proba
        else:
            res = self.regressor.predict_proba(dtest)
            res = pd.DataFrame(res)
            res.columns = self.regressor.classes_
            result['res_class'] = str(res.idxmax(axis=1).iloc[0])
            result['res_proba'] = res.max(axis=1).iloc[0]
        return result




    def predict1(self, test, debug=False):
        if isinstance(test, pd.Series):
            test = test.tolist()
        if isinstance(test, str) or isinstance(test, dict):
            test = [test]
            # return self.predict_single(test)

        test = [self.preprocessor.parse(node) for node in test]

        index = None
        if isinstance(test, pd.Series):
            index = test.index
            result = pd.DataFrame(columns=self.one_vs_all_models.keys(), index=index)
        else:
            result = pd.DataFrame(columns=self.one_vs_all_models.keys())

        test = self.preprocess_data(test)
        if debug:
            self.preprocessed_data = test
        test = self.preprocessor.process(test)
        if debug:
            self.processed_data = test

        test_tfidf = self.vectorizer.transform(test)
        if self.regressor == xgb:
            dtest = xgb.DMatrix(test_tfidf)

        if type(self.regressor).__name__ in ['type', 'module']:
            for class_name in self.one_vs_all_models:
                model = self.one_vs_all_models[class_name]
                if self.regressor == xgb:
                    proba = model.predict(dtest)
                else:
                    proba = model.predict(test_tfidf)
                result[class_name.apply(str)] = proba.apply(str)
            # return {'res_class': result.idxmax(axis=1).apply(str), 'res_proba':result.apply(str)}
            if result.shape[0] == 1:
                return {'res_class': str(result.idxmax(axis=1).iloc[0]), 'res_proba': float(result.max(axis=1).iloc[0])}
            else:
                return {'res_class': result.idxmax(axis=1).apply(str), 'res_proba': result.max(axis=1).apply(float)}
        else:
            y_pred = self.regressor.predict_proba(test_tfidf)
#             result = clf.predict_proba(X_full_arr)
            result = pd.DataFrame(y_pred)
            result.columns = self.regressor.classes_
            if index is not None:
                # print('apply index')
                result.index = index
            # print(result.idxmax(axis=1), result.max(axis=1))
            # return result.idxmax(axis=1), result
            if result.shape[0] == 1:
                return {'res_class': str(result.idxmax(axis=1).iloc[0]), 'res_proba': float(result.max(axis=1).iloc[0])}
            else:
                return {'res_class': result.idxmax(axis=1).apply(str), 'res_proba': result.max(axis=1).apply(float)}

    def set_ms(self, mystem):
        self.preprocessor.set_ms(mystem)

    def dump(self, path):
#        self.set_ms(None)
        preprocessed_data = self.preprocessed_data
        self.preprocessed_data = None
        processed_data = self.processed_data
        self.processed_data = None
        self.preprocessor.dump()
        with open(path, 'wb') as model:
            pickle.dump(self, model)

    def load(path):
        with open(path, 'rb') as model:
            tmp_model = pickle.load(model)
#        tmp_model.set_ms(MS)
        tmp_model.preprocessor.load()
        return tmp_model

    def reload(self, path):
        self = self.load(path)

    def get_debug_data(self):
        try:
            return {'preprocessed_data': self.preprocessed_data, 'processed_data': self.processed_data}
        except:
            return {'preprocessed_data': None, 'processed_data': None}

    


def text_proc_3(text):
    if text != '':
        text = re.sub(r"\s+", " ", text) # убрал лишние пробелы (\s+ Любой пробельный символ 1 и более вхождений шаблона)
        text = re.sub(r"^\s+", "", text) #  уберу первые пробелы в начале строки
        text = text.rstrip() #  уберу последний пробел в начале строки
    return text

def concat(vec):
    return ' '.join(vec)


def got_counter(per):
    counter = Counter()
    for i in range(len(per)):
        word = per[i]
        counter[word] += 1
    return counter

### Загрузка данных

In [ ]:
path = 'D:/ML/vIFO/datas/'
SS = 'way4_05'

data_all = pd.read_csv(path + SS + '.csv', sep=';', encoding = 'cp1251')
data_all.fillna('',inplace=True)

data_all = data_all[['OPEN_TIME', 'NUMBER', 'AFFECTED_ITEM_NAME', 'ASSIGNEE_NAME',
       'ASSIGNMENT_NAME', 'TITLE', 'PROTOCOL', 'DESCRIPTION', 'RESOLUTION', 'OPTIONS', 'Class']]

counter = got_counter(data_all['AFFECTED_ITEM_NAME'])
res_df = pd.DataFrame.from_dict(counter, orient='index').reset_index().rename(columns={'index':'event', 0:'count'})
res_df = res_df.sort_values(by=['count'], ascending=False).reset_index(drop=True)


#                                                event  count
#0   Way4 - Процессинг банковских карт (CI00058876)   4222
#1   Smart Vista - Фронтальная система терминальных...   2232
#2   Информационно-поисковая система для банковских...    865
#3   Система мониторинга сети банкоматов (СМСБ) ЦА,...    792
#4   Сверка операций на устройствах самообслуживани...    226
#5   Платформа автоматизации развития устройств сам...    156
#6   Smart Vista Fraud Prevention (CI00163953)    112
#7   Управление безопасностью ИТ-услуг (CI00306371)     99
#8                                                   81
#9   Формирование отчетности для клиентов банка (Вы...     40
#10  ФЛ.02_Расчеты по банковским картам и устройств...     19


data_all = data_all[data_all['Class'] != '' ].copy().reset_index(drop=True)


Name = pd.read_csv(path+'dict/mynames2.csv', sep=';', encoding = 'cp1251')
Name = [x for x in Name.iloc[:,0]]; Name = list(set(Name)); Name.sort()
Surname = pd.read_csv(path+'dict/mysurnames2.csv', sep=';', encoding = 'cp1251')
Surname = [x for x in Surname.iloc[:,0]]; Surname = list(set(Surname)); Surname.sort()
Secondname = pd.read_csv(path+'dict/mysecondnames2.csv', sep=';', encoding = 'cp1251')
Secondname = [x for x in Secondname.iloc[:,0]]; Secondname = list(set(Secondname)); Secondname.sort()


### Предобработка текста

In [ ]:
preprocessor = myPreprocessor(_names= [Name, Surname, Secondname])


data_all.columns
data_all['DESCRIPTION_'] = preprocessor.preprocess('DESCRIPTION', data_all['DESCRIPTION'])
data_all['OPTIONS_'] = preprocessor.preprocess('options', data_all['OPTIONS'])
data_all['RESOLUTION_'] = preprocessor.preprocess('DESCRIPTION', data_all['RESOLUTION'])

feat = ['DESCRIPTION_', 'OPTIONS_', 'RESOLUTION_']
S = 'process'
data_all[S] = [text_proc_3(concat([z for z in data_all.loc[x, feat]])) for x in tqdm(range(len(data_all[feat])))]     
data_all['NUMBER'] = data_all['NUMBER'].apply(lambda x: x.strip())
data_all['process'] = data_all['process'].apply(lambda x: x.strip())
data_all['Class'] = data_all['Class'].apply(lambda x: x.strip())
data_all = data_all[data_all['Class'] != ''].copy().reset_index(drop=True)

### Мешок слов и модель

In [ ]:
X_full = data_all['process']
y_full = data_all['Class']
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full,train_size=0.80, random_state=1, stratify = y_full)

X_train_c = X_train.copy()
X_test_c = X_test.copy() 
y_train_c  = [x for x in y_train]
y_test_c = [x for x in y_test]

vectorizer=TfidfVectorizer(analyzer='word', lowercase=False, ngram_range=(1, 3), min_df=0.0005, max_df=0.995, max_features = None)
X_full = vectorizer.fit_transform(X_full)
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

features = vectorizer.get_feature_names()
print("Количество признаков {}".format(X_train.shape[1]))

class_names = sorted(np.unique(y_full))
weights = [1 for i in range(0,len(class_names))]

for i in range(len(class_names)):
    weights[i] = len(y_full[y_full == class_names[i]])*100.0/len(y_full)

class_weight=dict(zip(class_names, weights))
class_weight


name = 'Logistic Regression'
clf_default = LogisticRegression(random_state=random_state, n_jobs=-1)
params = {'penalty' : ['l2'],
          'tol': [1e-2,1e-1],
          'C': [x for x in range(10, 21, 2)],
          'fit_intercept': (True, False),
          'class_weight': ['balanced', None, class_weight],
          'multi_class' : ['ovr'],
          'solver': ['sag'],
          'max_iter' : (5,7,9,11,15,21)
           }

start_time = datetime.datetime.now() 
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
clf = GridSearchCV(clf_default, params, cv=skf, n_jobs=-1, verbose = 10, error_score = 0)
clf.fit(X_train, y_train)
print(clf.best_params_)
print(datetime.datetime.now() - start_time)
best_clf = clf.best_estimator_
print(clf.best_score_)
print(best_clf)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
cv_score = cross_val_score(best_clf, X_train, y_train, cv = skf, n_jobs=-1)
print('cv_score.mean', cv_score.mean())
y_pred = best_clf.predict(X_test)
print ( 'Accuracy test', accuracy_score(y_test, y_pred))


cnf_matrix = confusion_matrix(y_test, best_clf.predict(X_test))
np.set_printoptions(precision=2)
fig = plt.figure()
fig.set_figheight(15)
fig.set_figwidth(15)
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True, title='Normalized confusion matrix test')
fig.savefig('lr_test1.jpg')

y_pred = best_clf.predict(X_full)
data_all['Class1'] = y_pred
mistake = [compare(data_all['Class'][x], data_all['Class1'][x]) for x in range(len(data_all))]
print(np.sum(mistake)/len(mistake))

### Анализ модели

In [ ]:

y_pred = best_clf.predict(X_test)
vct = pd.Series([x for x in y_test])
d = custom_confusion_matrix(vct, y_pred)
cm = confusion_matrix(vct, y_pred)
FP = cm.sum(axis=0) - np.diag(cm)
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)
TPR = TP*1.0/(TP+FN); TNR = TN*1.0/(TN+FP) 
PPV = TP*1.0/(TP+FP); NPV = TN*1.0/(TN+FN)
FPR = FP*1.0/(FP+TN); FNR = FN*1.0/(TP+FN)
FDR = FP*1.0/(TP+FP); ACC = (TP+TN)*1.0/(TP+FP+FN+TN)

columns = ['class', 'count', 'TP', 'FP', 'FN', 'TN', 'Полнота', 'TNR', 'precision', 'NPV', 'FPR', 'FNR', 'FDR', 'ACC']
matrix = pd.DataFrame( np.zeros( (len(vct.value_counts()),14) , dtype = float), columns = columns)
matrix['class'] = [x for x in d.index[:-1]]
matrix['count'] = [len(vct[vct == cls]) for cls in [x for x in d.index[:-1]] ]
matrix['TP'] = TP; matrix['FP'] = FP
matrix['TN'] = TN; matrix['FN'] = FN
matrix['Полнота'] = TPR; 
matrix['TNR'] = TNR
matrix['Точность'] = PPV; 
matrix['NPV'] = NPV
matrix['FPR'] = FPR; matrix['FNR'] = FNR
matrix['FDR'] = FDR; matrix['ACC'] = ACC

matrix.head(20)